In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import pandas as pd 
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm
import random

from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import LSTM, Dense, Dropout, Activation, Bidirectional
from tensorflow.keras.optimizers import Adam 

from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error, mean_squared_error
from tensorflow.keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Adam, Adamax, Nadam

import datetime

import warnings
warnings.filterwarnings('ignore')

from mealpy import FloatVar, IntegerVar,PermutationVar, Problem,MixedSetVar
from mealpy.swarm_based.GWO import OriginalGWO
from mealpy import PSO
from datetime import datetime
from mealpy import Problem, FloatVar, BBO 

In [2]:
def create_dataset(dataset, lookback=1):
    X, y = [], []
    for i in range(len(dataset) - lookback):
        sample = dataset[i:(i + lookback)]
        X.append(sample)
        y.append(dataset[i + lookback])
    return np.array(X), np.array(y)

In [3]:
df = pd.read_csv('hourly_res_data.csv').drop(['6-MF3MCLCBAT107', '6-MF3MCLCBAT108'], axis=1)

cols = list(df.columns[1:-1])
scaler = MinMaxScaler()
df[cols] = scaler.fit_transform(df[cols])

df_train = df[df['year'] <= 2021]
df_valid = df[df['year'] == 2022]
df_test = df[df['year'] == 2023]

df_train = df_train.drop(['DateTime', 'year'], axis=1)
df_valid = df_valid.drop(['DateTime', 'year'], axis=1)
df_test = df_test.drop(['DateTime', 'year'], axis=1)

print(f'Shape of Train Set: {df_train.shape}')
print(f'Shape of Valid Set: {df_valid.shape}')
print(f'Shape of Test Set: {df_test.shape}')

SEQUENCE_SIZE = 96
target_window = 24

def create_sequence(sequence_length, target_window, scaled_data):
    target_column_name = '1-MF3MCLCBAT101'
    target_column_index = list(scaled_data.columns).index(target_column_name)
    x = []
    y = []
    for i in tqdm(range(len(scaled_data) - sequence_length - target_window + 1)):
        sequence = scaled_data.iloc[i:i+sequence_length, :].values
        target = scaled_data.iloc[i+sequence_length:i+sequence_length+target_window, 
                                  target_column_index].values
        x.append(sequence)
        y.append(target)
    return np.array(x), np.array(y)

X_train, y_train = create_sequence(SEQUENCE_SIZE, target_window, df_train)
X_val, y_val = create_sequence(SEQUENCE_SIZE, target_window, df_valid)
X_test, y_test = create_sequence(SEQUENCE_SIZE, target_window, df_test)

Shape of Train Set: (26447, 6)
Shape of Valid Set: (8760, 6)
Shape of Test Set: (7897, 6)


100%|██████████| 7778/7778 [00:00<00:00, 9354.21it/s]


In [4]:
def model_training_architecture(lr,batch_sz,opt):
    model = Sequential()
    model.add(Bidirectional(LSTM(256), input_shape=X_train.shape[1:]))  
    model.add(Dense(24))

    if opt == 'adam':
        model_opt = Adam(learning_rate=lr)
    elif opt == 'sgd':
        model_opt = SGD(learning_rate=lr)
    else:
        model_opt = RMSprop(learning_rate=lr)
      
    model.compile(loss='mse', optimizer=model_opt)

      
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

    print('Model training ....')
    history = model.fit(X_train, y_train,verbose = False,
                        epochs=100, batch_size=batch_sz,
                        validation_data=(X_val, y_val),
                        callbacks=[early_stopping])
    
    model_evaluation = model.evaluate(X_test, y_test)
    print('Model eval ....')
    y_pred = model.predict(X_test)
    
    rsme_val = root_mean_squared_error(y_test, y_pred)
    print(rsme_val)
    
    return rsme_val

In [5]:
class MyProblem(Problem):
    def __init__(self, bounds=None, minmax="min", name="ml",date_time_str ="date", data=None,opt='opt', **kwargs):
        self.name = name
        self.data = data
        self.opt = opt
        super().__init__(bounds, minmax, **kwargs)
        
    def obj_func(self,x):
        x = self.decode_solution(x)
        lr = x["lr"]
        batch_sz = x["batch_sz"]
        opt = x["opt"]
        print([lr,batch_sz,opt])
        return model_training_architecture(lr,batch_sz,opt)

In [ ]:
now = datetime.now()
date_time_str = now.strftime("%Hh_%d_%m_%Y")


bound =  [MixedSetVar(valid_sets=(0.005,0.001,0.0005,0.0001), name="lr"),
          MixedSetVar(valid_sets=(16,32,64,128), name="batch_sz"),
          MixedSetVar(valid_sets=('adam', 'sdg','rmsprop'), name="opt")
         ]
epoch = 10
pop_size = 10

problem = MyProblem(bounds=bound, minmax="min", name="ml",data= None, opt = "pso", 
                log_to="file", log_file=f"pos-results_single_{date_time_str}.txt")
pso= PSO.OriginalPSO(epoch, pop_size)
p_best = pso.solve(problem)
print(f"Solution: {p_best}")